# This is running the LeNet 5 models, one dataset folder at a time.  

The model takes the image data from a root folder (greyscale/gy. original(RBG)/og or Four-channel/fc and runs them through a classic LeNet 5 model.

VERSIONS

Outputs are to the 3 (_gy, _or, _fc) data folders which themselves reflect the quality of the data provided.
So EACH folder needs to be run separately.

PARAMETERS

It adjusts the data to the 3 datasizes used in the KNN and Random Forest Models (64x64,128x128,256x256)
The 3 image sizes go into the LeNet5, which has a fixed structure

The 2x2 grid search is batches and epoch sizes

The model uses 5-fold on the 80% 'training-validation' data.  The data includes 20% for final testing.

LOCALISATION

root_folders - these are the sources of data (right at the end of each code block)
Adjust as needed.

It is assumed that earlier models and pip's have been installed.

TIMING

The models (when run in series and with concurrent cores) ran in 51 + 53 + 54 = 2hr 28 mins.
This is indicative as depends on the resources available.

RESULTS

each model x imagesize combination generates an excel file with results and logs
These are saved in the related root folders.





Running 06.DataSet_gy GREYSCALE

In [1]:
import os
import numpy as np
import cv2
import time
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from datetime import datetime
import concurrent.futures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images
def load_images_from_folder(folder, target_size):
    images = []
    labels = []
    class_names = os.listdir(folder)
    filenames = []
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is not None:
                if len(img.shape) == 2:  # Grayscale image
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                elif img.shape[2] == 4:  # 4-channel image (e.g., RGBA)
                    img = img[:, :, :3]  # Discard the alpha channel

                img = cv2.resize(img, target_size)  # Resize to target_size
                images.append(img)
                labels.append(label)
                filenames.append(filename)
    return np.array(images), np.array(labels), class_names, filenames

# Function to preprocess images for neural network
def preprocess_images(X, target_size):
    X_resized = []
    for img in X:
        img_resized = cv2.resize(img, target_size)
        X_resized.append(img_resized)
    X_resized = np.array(X_resized).astype('float32') / 255.0
    return X_resized

# Define LeNet-5 model
def create_lenet5(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(6, (5, 5), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define parameter grid for GridSearch
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

image_sizes = [(64, 64), (128, 128), (256, 256)]

def run_model(train_dataset_folder, test_dataset_folder, model_type, root_folder_suffix):
    model_start_time = datetime.now()
    log = []
    log.append(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    start_time = time.time()
    X_train_orig, y_train, class_names, train_filenames = load_images_from_folder(train_dataset_folder, target_size=(128, 128))
    y_train = to_categorical(y_train, num_classes=len(class_names))
    load_preprocess_time = time.time() - start_time
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")

    best_params = None
    best_score = 0
    best_target_size = None
    best_conf_matrix = None
    best_X_train_resized = None

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    timing_results = []
    cm_results = []

    for target_size in image_sizes:
        start_size_time = time.time()
        log.append(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")
        print(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")

        X_train_resized = preprocess_images(X_train_orig, target_size)
        input_shape = (target_size[0], target_size[1], 3)

        for batch_size in param_grid['batch_size']:
            for epochs in param_grid['epochs']:
                params = {
                    'batch_size': batch_size,
                    'epochs': epochs
                }

                lenet5 = create_lenet5(input_shape, len(class_names))

                start_grid_item_time = time.time()
                scores = []

                for train_index, val_index in kf.split(X_train_resized):
                    X_train_fold, X_val_fold = X_train_resized[train_index], X_train_resized[val_index]
                    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

                    lenet5.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0)
                    score = lenet5.evaluate(X_val_fold, y_val_fold, verbose=0)[1]
                    scores.append(score)

                mean_score = np.mean(scores)
                grid_item_time = time.time() - start_grid_item_time

                timing_results.append({
                    'image_size': target_size,
                    'batch_size': batch_size,
                    'epochs': epochs,
                    'mean_score': mean_score,
                    'time': grid_item_time,
                    'scoring_metric': 'accuracy'
                })

                log.append(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")
                print(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")

                X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)
                lenet5.fit(X_train_split, y_train_split, batch_size=batch_size, epochs=epochs, verbose=0)
                y_valid_pred = np.argmax(lenet5.predict(X_valid_split), axis=1)
                y_valid_true = np.argmax(y_valid_split, axis=1)
                cm = confusion_matrix(y_valid_true, y_valid_pred)
                cm_results.append({
                    'image_size': target_size,
                    'params': params,
                    'confusion_matrix': cm
                })

                if mean_score > best_score:
                    best_score = mean_score
                    best_params = params
                    best_target_size = target_size
                    best_conf_matrix = cm
                    best_X_train_resized = X_train_resized

        size_time = time.time() - start_size_time
        log.append(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")
        print(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    result_folder = os.path.dirname(train_dataset_folder)
    file_path = os.path.join(result_folder, f"{root_folder_suffix}_{model_type}_LeNet5_results_{timestamp}.xlsx")

    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        best_params_df = pd.DataFrame([best_params])
        best_params_df['image_size'] = [best_target_size]
        best_params_df.to_excel(writer, sheet_name='Best_Parameters', index=False)

        timing_df = pd.DataFrame(timing_results)
        timing_df.to_excel(writer, sheet_name='Timing_Results', index=False)

        cm_data = []
        for cm_result in cm_results:
            for i in range(len(class_names)):
                for j in range(len(class_names)):
                    cm_data.append({
                        'image_size': cm_result['image_size'],
                        'params': cm_result['params'],
                        'true_class': class_names[i],
                        'predicted_class': class_names[j],
                        'count': cm_result['confusion_matrix'][i, j]
                    })
        cm_df = pd.DataFrame(cm_data)
        cm_df.to_excel(writer, sheet_name='Confusion_Matrices', index=False)

    log.append(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    X_test_orig, y_test, _, test_filenames = load_images_from_folder(test_dataset_folder, target_size=best_target_size)
    y_test = to_categorical(y_test, num_classes=len(class_names))

    X_test_resized = preprocess_images(X_test_orig, best_target_size)

    start_time = time.time()
    final_model = create_lenet5((best_target_size[0], best_target_size[1], 3), len(class_names))
    final_model.fit(best_X_train_resized, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=0)
    train_time = time.time() - start_time

    start_time = time.time()
    y_test_pred = np.argmax(final_model.predict(X_test_resized), axis=1)
    y_test_true = np.argmax(y_test, axis=1)
    predict_time = time.time() - start_time

    test_accuracy = accuracy_score(y_test_true, y_test_pred)
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test_true, y_test_pred, target_names=class_names)}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test_true, y_test_pred)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test_true, y_test_pred, target_names=class_names)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test_true, y_test_pred)}")

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        test_report = classification_report(y_test_true, y_test_pred, target_names=class_names, output_dict=True)
        test_report_df = pd.DataFrame(test_report).transpose()
        test_report_df.to_excel(writer, sheet_name='Test_Classification_Report')

        test_cm = confusion_matrix(y_test_true, y_test_pred)
        test_cm_df = pd.DataFrame(test_cm, index=class_names, columns=class_names)
        test_cm_df.to_excel(writer, sheet_name='Test_Confusion_Matrix')

        predictions_df = pd.DataFrame({
            'Filename': test_filenames,
            'Actual': [class_names[label] for label in y_test_true],
            'Predicted': [class_names[label] for label in y_test_pred]
        })
        predictions_df.to_excel(writer, sheet_name='Image_Predictions', index=False)

        summary_data = {
            'Best Score': [best_score],
            'Best Parameters': [best_params],
            'Best Image Size': [best_target_size],
            'Test Accuracy': [test_accuracy],
            'Training Time': [train_time],
            'Prediction Time': [predict_time]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)

        log_df = pd.DataFrame(log, columns=['Log'])
        log_df.to_excel(writer, sheet_name='Log', index=False)

    model_end_time = datetime.now()
    log.append(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")

# Function to iterate over the data directories and run the model
def run_models_for_all_datasets(root_folders):
    overall_start_time = datetime.now()
    print(f"Overall processing started at {overall_start_time}")

    model_types = ['country', 'exact_piece', 'force', 'piece']

    tasks = []

    for root_folder in root_folders:
        root_folder_suffix = os.path.basename(root_folder)[-2:]
        for model_type in model_types:
            train_folder = os.path.join(root_folder, f"{model_type}_train")
            test_folder = os.path.join(root_folder, f"{model_type}_test")

            if os.path.exists(train_folder) and os.path.exists(test_folder):
                tasks.append((train_folder, test_folder, model_type, root_folder_suffix))
            else:
                print(f"Skipping {model_type} in {root_folder} as train or test folder does not exist")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(run_model, train, test, model, suffix) for train, test, model, suffix in tasks]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    overall_end_time = datetime.now()
    print(f"Overall processing ended at {overall_end_time}")
    print(f"Total duration for all models: {overall_end_time - overall_start_time}")

root_folders = [
    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\06.DataSets_gy',
#    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\07.DataSets_or',
#    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\08.DataSets_fc'
]

run_models_for_all_datasets(root_folders)


Overall processing started at 2024-08-05 22:37:31.267233
Processing gy:country - Started at 2024-08-05 22:37:31.268232
Processing gy:country - Loading and preprocessing training data
Processing gy:exact_piece - Started at 2024-08-05 22:37:31.268232
Processing gy:exact_piece - Loading and preprocessing training data
Processing gy:force - Started at 2024-08-05 22:37:31.269232
Processing gy:force - Loading and preprocessing training data
Processing gy:piece - Started at 2024-08-05 22:37:31.269232
Processing gy:piece - Loading and preprocessing training data
Processing gy:piece - Time taken for loading and preprocessing: 31.00 seconds
Processing gy:piece - Evaluating for image size: (64, 64)
Processing gy:country - Time taken for loading and preprocessing: 31.15 seconds
Processing gy:country - Evaluating for image size: (64, 64)
Processing gy:force - Time taken for loading and preprocessing: 31.20 seconds
Processing gy:force - Evaluating for image size: (64, 64)
Processing gy:exact_piece -

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Processing gy:piece - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9282 - Time: 19.68 seconds
Processing gy:country - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.8645 - Time: 19.74 seconds
Processing gy:force - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9535 - Time: 20.06 seconds
Processing gy:exact_piece - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.6865 - Time: 20.13 seconds
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step
Processing gy:exact_piece - Total time taken for image size (256, 256): 2238.13 seconds

Processing gy:exact_piece - Best Score: 0.9444897890090942
Processing gy:exact_piece - Best Parameters: {'batch_size': 64, 'epochs': 20}
Processing gy:exact_piece - Best Image Size: (256, 256)
Processing gy:exact_piece - Grid search results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\06.DataSets_gy\gy_exact_piece_LeNet5_results_20240805_232645.xlsx
Processing gy:exact_piece - Loading and preprocessing test data
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
Processing gy:country - Total time taken for image size (256, 256): 2242.42 seconds

Processing gy:country - Best Score: 0.9567346930503845
Processing gy:country - Best Parameters: {'batch_size': 32, 'epochs': 20}
Processing gy:country - Best Image Size: (128, 128)
Processing gy:country - Grid search results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Project

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step
Processing gy:force - Test Accuracy: 0.8824
Processing gy:force - Time taken for final training: 67.19 seconds
Processing gy:force - Time taken for final prediction: 1.51 seconds
Processing gy:force - Test Classification Report:
              precision    recall  f1-score   support

         AIR       0.86      0.72      0.78        68
         LND       0.98      0.76      0.86        68
         SEA       0.86      0.99      0.92       170

    accuracy                           0.88       306
   macro avg       0.90      0.83      0.86       306
weighted avg       0.89      0.88      0.88       306

Processing gy:force - Test Confusion Matrix:
[[ 49   0  19]
 [  8  52   8]
 [  0   1 169]]
Processing gy:force - Ended at 2024-08-05 23:27:54.601513
Processing gy:force - Total duration: 0:50:23.332281
Processing gy:force - Best Parameters identified from the grid search: {'batch_size': 32, 'epochs': 10}
Processing gy:force - Best Image Size identi

C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packag

Processing gy:exact_piece - Ended at 2024-08-05 23:28:47.268125
Processing gy:exact_piece - Total duration: 0:51:15.999893
Processing gy:exact_piece - Best Parameters identified from the grid search: {'batch_size': 64, 'epochs': 20}
Processing gy:exact_piece - Best Image Size identified from the grid search: (256, 256)
Processing gy:exact_piece - Final test results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\06.DataSets_gy\gy_exact_piece_LeNet5_results_20240805_232645.xlsx
Overall processing ended at 2024-08-05 23:28:47.422871
Total duration for all models: 0:51:16.155638


Running 06.DataSet_or ORIGINAL

In [2]:
import os
import numpy as np
import cv2
import time
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from datetime import datetime
import concurrent.futures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images
def load_images_from_folder(folder, target_size):
    images = []
    labels = []
    class_names = os.listdir(folder)
    filenames = []
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is not None:
                if len(img.shape) == 2:  # Grayscale image
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                elif img.shape[2] == 4:  # 4-channel image (e.g., RGBA)
                    img = img[:, :, :3]  # Discard the alpha channel

                img = cv2.resize(img, target_size)  # Resize to target_size
                images.append(img)
                labels.append(label)
                filenames.append(filename)
    return np.array(images), np.array(labels), class_names, filenames

# Function to preprocess images for neural network
def preprocess_images(X, target_size):
    X_resized = []
    for img in X:
        img_resized = cv2.resize(img, target_size)
        X_resized.append(img_resized)
    X_resized = np.array(X_resized).astype('float32') / 255.0
    return X_resized

# Define LeNet-5 model
def create_lenet5(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(6, (5, 5), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define parameter grid for GridSearch
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

image_sizes = [(64, 64), (128, 128), (256, 256)]

def run_model(train_dataset_folder, test_dataset_folder, model_type, root_folder_suffix):
    model_start_time = datetime.now()
    log = []
    log.append(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    start_time = time.time()
    X_train_orig, y_train, class_names, train_filenames = load_images_from_folder(train_dataset_folder, target_size=(128, 128))
    y_train = to_categorical(y_train, num_classes=len(class_names))
    load_preprocess_time = time.time() - start_time
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")

    best_params = None
    best_score = 0
    best_target_size = None
    best_conf_matrix = None
    best_X_train_resized = None

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    timing_results = []
    cm_results = []

    for target_size in image_sizes:
        start_size_time = time.time()
        log.append(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")
        print(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")

        X_train_resized = preprocess_images(X_train_orig, target_size)
        input_shape = (target_size[0], target_size[1], 3)

        for batch_size in param_grid['batch_size']:
            for epochs in param_grid['epochs']:
                params = {
                    'batch_size': batch_size,
                    'epochs': epochs
                }

                lenet5 = create_lenet5(input_shape, len(class_names))

                start_grid_item_time = time.time()
                scores = []

                for train_index, val_index in kf.split(X_train_resized):
                    X_train_fold, X_val_fold = X_train_resized[train_index], X_train_resized[val_index]
                    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

                    lenet5.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0)
                    score = lenet5.evaluate(X_val_fold, y_val_fold, verbose=0)[1]
                    scores.append(score)

                mean_score = np.mean(scores)
                grid_item_time = time.time() - start_grid_item_time

                timing_results.append({
                    'image_size': target_size,
                    'batch_size': batch_size,
                    'epochs': epochs,
                    'mean_score': mean_score,
                    'time': grid_item_time,
                    'scoring_metric': 'accuracy'
                })

                log.append(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")
                print(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")

                X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)
                lenet5.fit(X_train_split, y_train_split, batch_size=batch_size, epochs=epochs, verbose=0)
                y_valid_pred = np.argmax(lenet5.predict(X_valid_split), axis=1)
                y_valid_true = np.argmax(y_valid_split, axis=1)
                cm = confusion_matrix(y_valid_true, y_valid_pred)
                cm_results.append({
                    'image_size': target_size,
                    'params': params,
                    'confusion_matrix': cm
                })

                if mean_score > best_score:
                    best_score = mean_score
                    best_params = params
                    best_target_size = target_size
                    best_conf_matrix = cm
                    best_X_train_resized = X_train_resized

        size_time = time.time() - start_size_time
        log.append(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")
        print(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    result_folder = os.path.dirname(train_dataset_folder)
    file_path = os.path.join(result_folder, f"{root_folder_suffix}_{model_type}_LeNet5_results_{timestamp}.xlsx")

    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        best_params_df = pd.DataFrame([best_params])
        best_params_df['image_size'] = [best_target_size]
        best_params_df.to_excel(writer, sheet_name='Best_Parameters', index=False)

        timing_df = pd.DataFrame(timing_results)
        timing_df.to_excel(writer, sheet_name='Timing_Results', index=False)

        cm_data = []
        for cm_result in cm_results:
            for i in range(len(class_names)):
                for j in range(len(class_names)):
                    cm_data.append({
                        'image_size': cm_result['image_size'],
                        'params': cm_result['params'],
                        'true_class': class_names[i],
                        'predicted_class': class_names[j],
                        'count': cm_result['confusion_matrix'][i, j]
                    })
        cm_df = pd.DataFrame(cm_data)
        cm_df.to_excel(writer, sheet_name='Confusion_Matrices', index=False)

    log.append(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    X_test_orig, y_test, _, test_filenames = load_images_from_folder(test_dataset_folder, target_size=best_target_size)
    y_test = to_categorical(y_test, num_classes=len(class_names))

    X_test_resized = preprocess_images(X_test_orig, best_target_size)

    start_time = time.time()
    final_model = create_lenet5((best_target_size[0], best_target_size[1], 3), len(class_names))
    final_model.fit(best_X_train_resized, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=0)
    train_time = time.time() - start_time

    start_time = time.time()
    y_test_pred = np.argmax(final_model.predict(X_test_resized), axis=1)
    y_test_true = np.argmax(y_test, axis=1)
    predict_time = time.time() - start_time

    test_accuracy = accuracy_score(y_test_true, y_test_pred)
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test_true, y_test_pred, target_names=class_names)}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test_true, y_test_pred)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test_true, y_test_pred, target_names=class_names)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test_true, y_test_pred)}")

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        test_report = classification_report(y_test_true, y_test_pred, target_names=class_names, output_dict=True)
        test_report_df = pd.DataFrame(test_report).transpose()
        test_report_df.to_excel(writer, sheet_name='Test_Classification_Report')

        test_cm = confusion_matrix(y_test_true, y_test_pred)
        test_cm_df = pd.DataFrame(test_cm, index=class_names, columns=class_names)
        test_cm_df.to_excel(writer, sheet_name='Test_Confusion_Matrix')

        predictions_df = pd.DataFrame({
            'Filename': test_filenames,
            'Actual': [class_names[label] for label in y_test_true],
            'Predicted': [class_names[label] for label in y_test_pred]
        })
        predictions_df.to_excel(writer, sheet_name='Image_Predictions', index=False)

        summary_data = {
            'Best Score': [best_score],
            'Best Parameters': [best_params],
            'Best Image Size': [best_target_size],
            'Test Accuracy': [test_accuracy],
            'Training Time': [train_time],
            'Prediction Time': [predict_time]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)

        log_df = pd.DataFrame(log, columns=['Log'])
        log_df.to_excel(writer, sheet_name='Log', index=False)

    model_end_time = datetime.now()
    log.append(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")

# Function to iterate over the data directories and run the model
def run_models_for_all_datasets(root_folders):
    overall_start_time = datetime.now()
    print(f"Overall processing started at {overall_start_time}")

    model_types = ['country', 'exact_piece', 'force', 'piece']

    tasks = []

    for root_folder in root_folders:
        root_folder_suffix = os.path.basename(root_folder)[-2:]
        for model_type in model_types:
            train_folder = os.path.join(root_folder, f"{model_type}_train")
            test_folder = os.path.join(root_folder, f"{model_type}_test")

            if os.path.exists(train_folder) and os.path.exists(test_folder):
                tasks.append((train_folder, test_folder, model_type, root_folder_suffix))
            else:
                print(f"Skipping {model_type} in {root_folder} as train or test folder does not exist")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(run_model, train, test, model, suffix) for train, test, model, suffix in tasks]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    overall_end_time = datetime.now()
    print(f"Overall processing ended at {overall_end_time}")
    print(f"Total duration for all models: {overall_end_time - overall_start_time}")

root_folders = [
#    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\06.DataSets_gy',
    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\07.DataSets_or',
#    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\08.DataSets_fc'
]

run_models_for_all_datasets(root_folders)

Overall processing started at 2024-08-05 23:31:21.364193
Processing or:country - Started at 2024-08-05 23:31:21.366177
Processing or:country - Loading and preprocessing training data
Processing or:exact_piece - Started at 2024-08-05 23:31:21.366683
Processing or:exact_piece - Loading and preprocessing training data
Processing or:force - Started at 2024-08-05 23:31:21.366683
Processing or:force - Loading and preprocessing training data
Processing or:piece - Started at 2024-08-05 23:31:21.366683
Processing or:piece - Loading and preprocessing training data
Processing or:piece - Time taken for loading and preprocessing: 46.67 seconds
Processing or:piece - Evaluating for image size: (64, 64)
Processing or:country - Time taken for loading and preprocessing: 46.70 seconds
Processing or:country - Evaluating for image size: (64, 64)
Processing or:force - Time taken for loading and preprocessing: 46.70 seconds
Processing or:force - Evaluating for image size: (64, 64)
Processing or:exact_piece -

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Processing or:force - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9894 - Time: 25.35 seconds
Processing or:exact_piece - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.8000 - Time: 25.74 seconds
Processing or:country - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.8547 - Time: 26.06 seconds
Processing or:piece - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9282 - Time: 26.20 seconds
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step 

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
Processing or:force - Test Accuracy: 0.9804
Processing or:force - Time taken for final training: 43.97 seconds
Processing or:force - Time taken for final prediction: 1.58 seconds
Processing or:force - Test Classification Report:
              precision    recall  f1-score   support

         AIR       0.93      0.99      0.96        68
         LND       0.98      0.94      0.96        68
         SEA       1.00      0.99      1.00       170

    accuracy                           0.98       306
   macro avg       0.97      0.97      0.97       306
weighted avg       0.98      0.98      0.98       306

Processing or:force - Test Confusion Matrix:
[[ 67   1   0]
 [  4  64   0]
 [  1   0 169]]
Processing or:force - Ended at 2024-08-06 00:22:58.504343
Processing or:force - Total duration: 0:51:37.137660
Processing or:force - Best Parameters identified from the grid search: {'batch_size': 32, 'epochs': 20}
Processing or:force - Best Image Size identi

C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packag

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 174ms/step
Processing or:piece - Test Accuracy: 0.7810
Processing or:piece - Time taken for final training: 121.37 seconds
Processing or:piece - Time taken for final prediction: 1.95 seconds
Processing or:piece - Test Classification Report:
              precision    recall  f1-score   support

       01INF       0.97      0.88      0.92        34
       02TNK       0.84      0.79      0.82        34
       03FGT       0.77      0.71      0.74        34
       04BMB       0.60      0.76      0.68        34
       05DES       0.65      0.65      0.65        34
       06TRS       0.79      0.79      0.79        34
       07SUB       0.96      0.65      0.77        34
       08BAT       0.84      0.79      0.82        34
       09CAR       0.74      1.00      0.85        34

    accuracy                           0.78       306
   macro avg       0.80      0.78      0.78       306
weighted avg       0.80      0.78      0.78       306

Processing or:piece - Te

C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ReCas\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Processing or:piece - Ended at 2024-08-06 00:24:30.896340
Processing or:piece - Total duration: 0:53:09.529657
Processing or:piece - Best Parameters identified from the grid search: {'batch_size': 32, 'epochs': 20}
Processing or:piece - Best Image Size identified from the grid search: (256, 256)
Processing or:piece - Final test results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\07.DataSets_or\or_piece_LeNet5_results_20240806_002201.xlsx
Processing or:exact_piece - Ended at 2024-08-06 00:24:31.097185
Processing or:exact_piece - Total duration: 0:53:09.730502
Processing or:exact_piece - Best Parameters identified from the grid search: {'batch_size': 64, 'epochs': 20}
Processing or:exact_piece - Best Image Size identified from the grid search: (256, 256)
Processing or:exact_piece - Final test results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\07.DataSets_or\or_exact_piece_LeNet5_results_20240806_002158.xlsx
Overall proc

Running 06.DataSet_fc FOUR CHANNEL (i.e. original + depth map)

In [3]:
import os
import numpy as np
import cv2
import time
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from datetime import datetime
import concurrent.futures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images
def load_images_from_folder(folder, target_size):
    images = []
    labels = []
    class_names = os.listdir(folder)
    filenames = []
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is not None:
                if len(img.shape) == 2:  # Grayscale image
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                elif img.shape[2] == 4:  # 4-channel image (e.g., RGBA)
                    img = img[:, :, :3]  # Discard the alpha channel

                img = cv2.resize(img, target_size)  # Resize to target_size
                images.append(img)
                labels.append(label)
                filenames.append(filename)
    return np.array(images), np.array(labels), class_names, filenames

# Function to preprocess images for neural network
def preprocess_images(X, target_size):
    X_resized = []
    for img in X:
        img_resized = cv2.resize(img, target_size)
        X_resized.append(img_resized)
    X_resized = np.array(X_resized).astype('float32') / 255.0
    return X_resized

# Define LeNet-5 model
def create_lenet5(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(6, (5, 5), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define parameter grid for GridSearch
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

image_sizes = [(64, 64), (128, 128), (256, 256)]

def run_model(train_dataset_folder, test_dataset_folder, model_type, root_folder_suffix):
    model_start_time = datetime.now()
    log = []
    log.append(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Started at {model_start_time}")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing training data")
    start_time = time.time()
    X_train_orig, y_train, class_names, train_filenames = load_images_from_folder(train_dataset_folder, target_size=(128, 128))
    y_train = to_categorical(y_train, num_classes=len(class_names))
    load_preprocess_time = time.time() - start_time
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for loading and preprocessing: {load_preprocess_time:.2f} seconds")

    best_params = None
    best_score = 0
    best_target_size = None
    best_conf_matrix = None
    best_X_train_resized = None

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    timing_results = []
    cm_results = []

    for target_size in image_sizes:
        start_size_time = time.time()
        log.append(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")
        print(f"Processing {root_folder_suffix}:{model_type} - Evaluating for image size: {target_size}")

        X_train_resized = preprocess_images(X_train_orig, target_size)
        input_shape = (target_size[0], target_size[1], 3)

        for batch_size in param_grid['batch_size']:
            for epochs in param_grid['epochs']:
                params = {
                    'batch_size': batch_size,
                    'epochs': epochs
                }

                lenet5 = create_lenet5(input_shape, len(class_names))

                start_grid_item_time = time.time()
                scores = []

                for train_index, val_index in kf.split(X_train_resized):
                    X_train_fold, X_val_fold = X_train_resized[train_index], X_train_resized[val_index]
                    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

                    lenet5.fit(X_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, verbose=0)
                    score = lenet5.evaluate(X_val_fold, y_val_fold, verbose=0)[1]
                    scores.append(score)

                mean_score = np.mean(scores)
                grid_item_time = time.time() - start_grid_item_time

                timing_results.append({
                    'image_size': target_size,
                    'batch_size': batch_size,
                    'epochs': epochs,
                    'mean_score': mean_score,
                    'time': grid_item_time,
                    'scoring_metric': 'accuracy'
                })

                log.append(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")
                print(f"Processing {root_folder_suffix}:{model_type} - Params: {params} - Mean Score (accuracy): {mean_score:.4f} - Time: {grid_item_time:.2f} seconds")

                X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)
                lenet5.fit(X_train_split, y_train_split, batch_size=batch_size, epochs=epochs, verbose=0)
                y_valid_pred = np.argmax(lenet5.predict(X_valid_split), axis=1)
                y_valid_true = np.argmax(y_valid_split, axis=1)
                cm = confusion_matrix(y_valid_true, y_valid_pred)
                cm_results.append({
                    'image_size': target_size,
                    'params': params,
                    'confusion_matrix': cm
                })

                if mean_score > best_score:
                    best_score = mean_score
                    best_params = params
                    best_target_size = target_size
                    best_conf_matrix = cm
                    best_X_train_resized = X_train_resized

        size_time = time.time() - start_size_time
        log.append(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")
        print(f"Processing {root_folder_suffix}:{model_type} - Total time taken for image size {target_size}: {size_time:.2f} seconds\n")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Score: {best_score}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size: {best_target_size}")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    result_folder = os.path.dirname(train_dataset_folder)
    file_path = os.path.join(result_folder, f"{root_folder_suffix}_{model_type}_LeNet5_results_{timestamp}.xlsx")

    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        best_params_df = pd.DataFrame([best_params])
        best_params_df['image_size'] = [best_target_size]
        best_params_df.to_excel(writer, sheet_name='Best_Parameters', index=False)

        timing_df = pd.DataFrame(timing_results)
        timing_df.to_excel(writer, sheet_name='Timing_Results', index=False)

        cm_data = []
        for cm_result in cm_results:
            for i in range(len(class_names)):
                for j in range(len(class_names)):
                    cm_data.append({
                        'image_size': cm_result['image_size'],
                        'params': cm_result['params'],
                        'true_class': class_names[i],
                        'predicted_class': class_names[j],
                        'count': cm_result['confusion_matrix'][i, j]
                    })
        cm_df = pd.DataFrame(cm_data)
        cm_df.to_excel(writer, sheet_name='Confusion_Matrices', index=False)

    log.append(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Grid search results saved to {file_path}")

    log.append(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    print(f"Processing {root_folder_suffix}:{model_type} - Loading and preprocessing test data")
    X_test_orig, y_test, _, test_filenames = load_images_from_folder(test_dataset_folder, target_size=best_target_size)
    y_test = to_categorical(y_test, num_classes=len(class_names))

    X_test_resized = preprocess_images(X_test_orig, best_target_size)

    start_time = time.time()
    final_model = create_lenet5((best_target_size[0], best_target_size[1], 3), len(class_names))
    final_model.fit(best_X_train_resized, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=0)
    train_time = time.time() - start_time

    start_time = time.time()
    y_test_pred = np.argmax(final_model.predict(X_test_resized), axis=1)
    y_test_true = np.argmax(y_test, axis=1)
    predict_time = time.time() - start_time

    test_accuracy = accuracy_score(y_test_true, y_test_pred)
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test_true, y_test_pred, target_names=class_names)}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test_true, y_test_pred)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Accuracy: {test_accuracy:.4f}")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final training: {train_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Time taken for final prediction: {predict_time:.2f} seconds")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Classification Report:\n{classification_report(y_test_true, y_test_pred, target_names=class_names)}")
    print(f"Processing {root_folder_suffix}:{model_type} - Test Confusion Matrix:\n{confusion_matrix(y_test_true, y_test_pred)}")

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        test_report = classification_report(y_test_true, y_test_pred, target_names=class_names, output_dict=True)
        test_report_df = pd.DataFrame(test_report).transpose()
        test_report_df.to_excel(writer, sheet_name='Test_Classification_Report')

        test_cm = confusion_matrix(y_test_true, y_test_pred)
        test_cm_df = pd.DataFrame(test_cm, index=class_names, columns=class_names)
        test_cm_df.to_excel(writer, sheet_name='Test_Confusion_Matrix')

        predictions_df = pd.DataFrame({
            'Filename': test_filenames,
            'Actual': [class_names[label] for label in y_test_true],
            'Predicted': [class_names[label] for label in y_test_pred]
        })
        predictions_df.to_excel(writer, sheet_name='Image_Predictions', index=False)

        summary_data = {
            'Best Score': [best_score],
            'Best Parameters': [best_params],
            'Best Image Size': [best_target_size],
            'Test Accuracy': [test_accuracy],
            'Training Time': [train_time],
            'Prediction Time': [predict_time]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)

        log_df = pd.DataFrame(log, columns=['Log'])
        log_df.to_excel(writer, sheet_name='Log', index=False)

    model_end_time = datetime.now()
    log.append(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    log.append(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")
    print(f"Processing {root_folder_suffix}:{model_type} - Ended at {model_end_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Total duration: {model_end_time - model_start_time}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Parameters identified from the grid search: {best_params}")
    print(f"Processing {root_folder_suffix}:{model_type} - Best Image Size identified from the grid search: {best_target_size}")
    print(f"Processing {root_folder_suffix}:{model_type} - Final test results saved to {file_path}")

# Function to iterate over the data directories and run the model
def run_models_for_all_datasets(root_folders):
    overall_start_time = datetime.now()
    print(f"Overall processing started at {overall_start_time}")

    model_types = ['country', 'exact_piece', 'force', 'piece']

    tasks = []

    for root_folder in root_folders:
        root_folder_suffix = os.path.basename(root_folder)[-2:]
        for model_type in model_types:
            train_folder = os.path.join(root_folder, f"{model_type}_train")
            test_folder = os.path.join(root_folder, f"{model_type}_test")

            if os.path.exists(train_folder) and os.path.exists(test_folder):
                tasks.append((train_folder, test_folder, model_type, root_folder_suffix))
            else:
                print(f"Skipping {model_type} in {root_folder} as train or test folder does not exist")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(run_model, train, test, model, suffix) for train, test, model, suffix in tasks]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    overall_end_time = datetime.now()
    print(f"Overall processing ended at {overall_end_time}")
    print(f"Total duration for all models: {overall_end_time - overall_start_time}")

root_folders = [
#    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\06.DataSets_gy',
#    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\07.DataSets_or',
    'C:\\Users\\ReCas\\OneDrive\\Documents\\2024_AIMachineLearning\\99_Projects\\08.DataSets_fc'
]

run_models_for_all_datasets(root_folders)

Overall processing started at 2024-08-06 00:24:31.299226
Processing fc:country - Started at 2024-08-06 00:24:31.301225
Processing fc:country - Loading and preprocessing training data
Processing fc:exact_piece - Started at 2024-08-06 00:24:31.301225
Processing fc:exact_piece - Loading and preprocessing training data
Processing fc:force - Started at 2024-08-06 00:24:31.301225
Processing fc:force - Loading and preprocessing training data
Processing fc:piece - Started at 2024-08-06 00:24:31.302224
Processing fc:piece - Loading and preprocessing training data
Processing fc:force - Time taken for loading and preprocessing: 63.99 seconds
Processing fc:force - Evaluating for image size: (64, 64)
Processing fc:piece - Time taken for loading and preprocessing: 64.05 seconds
Processing fc:piece - Evaluating for image size: (64, 64)
Processing fc:exact_piece - Time taken for loading and preprocessing: 64.11 seconds
Processing fc:exact_piece - Evaluating for image size: (64, 64)
Processing fc:count

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Processing fc:force - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9796 - Time: 22.70 seconds
Processing fc:piece - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9347 - Time: 22.73 seconds
Processing fc:country - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.9453 - Time: 23.19 seconds
Processing fc:exact_piece - Params: {'batch_size': 32, 'epochs': 10} - Mean Score (accuracy): 0.7779 - Time: 23.44 seconds
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


...

Processing fc:force - Params: {'batch_size': 64, 'epochs': 20} - Mean Score (accuracy): 0.9959 - Time: 597.11 seconds
Processing fc:country - Params: {'batch_size': 64, 'epochs': 20} - Mean Score (accuracy): 0.9747 - Time: 597.76 seconds
Processing fc:piece - Params: {'batch_size': 64, 'epochs': 20} - Mean Score (accuracy): 0.9804 - Time: 601.69 seconds
Processing fc:exact_piece - Params: {'batch_size': 64, 'epochs': 20} - Mean Score (accuracy): 0.9331 - Time: 611.61 seconds
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step
Processing fc:force - Total time taken for image size (256, 256): 2314.50 seconds

Processing fc:force - Best Score: 0.9959183812141419
Processing fc:force - Best Parameters: {'batch_size': 32, 'epochs': 20}
Processing fc:force - Best Image Size: (128, 128)
Processing fc:force - Grid search results saved to C:\Users\ReCas\OneDrive\Documents\2024_AIMachineLearning\99_Projects\08.DataSets_fc\fc_force_LeNet5_results_20240806_011531.xlsx
Processing fc:force - Loading and preprocess

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Processing fc:force - Test Accuracy: 0.9804
Processing fc:force - Time taken for final training: 14.64 seconds
Processing fc:force - Time taken for final prediction: 0.20 seconds
Processing fc:force - Test Classification Report:
              precision    recall  f1-score   support

         AIR       0.93      0.99      0.96        68
         LND       0.98      0.94      0.96        68
         SEA       1.00      0.99      1.00       170

    accuracy                           0.98       306
   macro avg       0.97      0.97      0.97       306
weighted avg       0.98      0.98      0.98       306

Processing fc:force - Test Confusion Matrix:
[[ 67   1   0]
 [  4  64   0]
 [  1   0 169]]
Processing fc:force - Ended at 2024-08-06 01:16:24.299322
Processing fc:force - Total duration: 0:51:52.998097
Processing fc:force - Best Parameters identified from the grid search: {'batch_size': 32, 'epochs': 20}
Processing fc:force - Best Image Size identi

C:\Users\ReCas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
Processing fc:country - Test Accuracy: 0.8464
Processing fc:country - Time taken for final training: 121.31 seconds
Processing fc:country - Time taken for final prediction: 1.12 seconds
Processing fc:country - Test Classification Report:
              precision    recall  f1-score   support

      100RUS       0.81      0.79      0.80        61
      200GER       0.74      0.95      0.83        61
       300UK       0.96      0.74      0.83        61
      400JAP       0.97      0.95      0.96        62
      500USA       0.80      0.80      0.80        61

    accuracy                           0.85       306
   macro avg       0.86      0.85      0.85       306
weighted avg       0.86      0.85      0.85       306

Processing fc:country - Test Confusion Matrix:
[[48 12  0  0  1]
 [ 3 58  0  0  0]
 [ 3  0 45  2 11]
 [ 3  0  0 59  0]
 [ 2  8  2  0 49]]
Processing fc:country - Ended at 2024-08-06 01:18:27.844525
Processing fc:country - Total durat